# Deep Agents

[Deep Agents](https://docs.langchain.com/oss/python/deepagents/overview) can be seen as DeepResearch produced by the LangChain team. This section tests whether Deep Agents has the ability to actively invoke search and gain deep insights by querying recent news.

> This is just a quick start. For more information, please visit the official website!

In [1]:
# !pip install deepagents dashscope

## 1. Load Model

In [2]:
import os
import dashscope

from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from deepagents import create_deep_agent
from dashscope import Generation

# Load model configuration
_ = load_dotenv()

# Configure api_key for DashScope
dashscope.api_key=os.getenv("DASHSCOPE_API_KEY")

# Load model
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="qwen3-max",
)

## 2. Create Search Tool

We use the `dashscope_search` search tool created in the previous section.

In [3]:
@tool
def dashscope_search(query: str) -> str:
    """Search the internet using Quark Search API"""
    response = Generation.call(
        model='qwen-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

## 3. Create Deep Agent

In [4]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """You are a senior researcher. Your job is to conduct comprehensive and in-depth research, and write a concise report.

You can use the internet search tool as the primary way to gather information.

## `dashscope_search`

Use this tool to search the internet for the specified query.
"""

@tool
def get_today_date() -> str:
    """Get today's date"""
    return datetime.now().strftime("%Y-%m-%d")

agent = create_deep_agent(
    model=llm,
    tools=[dashscope_search, get_today_date],
    system_prompt=research_instructions
)

## 4. Run Agent

Our question is:

```
Who is the newly appointed President of Bolivia? Please introduce this president.
```

Because recently (November 8, 2025) the Bolivian President `Rodrigo Paz Pereira` just took office, this information is definitely not in the pre-training data. This question can be used to verify whether Deep Agents actually connects to the internet.

In [5]:
result = agent.invoke({"messages": [
    {"role": "user", "content": "Who is the newly appointed President of Bolivia? Please introduce this president."}
]})

# Final response
print(result["messages"][-1].content)

# President Luis Arce of Bolivia

As of my latest information, **Luis Arce** is the current President of Bolivia, having taken office on November 8, 2020. There has been no newly appointed president in 2024.

## Background and Biography

**Luis Arce Catacora** was born on January 27, 1963, in La Paz, Bolivia, into a middle-class family—his father was a teacher and his mother a homemaker.

### Education and Early Career
- Studied economics at Universidad Mayor de San Andrés (UMSA) in La Paz
- Later pursued advanced studies in economic development and public finance
- Worked as an economic advisor for international organizations including the UNDP and Inter-American Development Bank

### Political Rise
- Appointed Minister of Economy and Public Finance by President Evo Morales in 2006
- Served in this role until 2019, implementing policies that reduced poverty and promoted economic growth
- Played a key role in nationalizing Bolivia's natural gas industry, which became a major government

In [6]:
# Thinking process
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

Who is the newly appointed President of Bolivia? Please introduce this president.
================================== Ai Message ==================================

I'll help you find information about the newly appointed President of Bolivia. Let me search for the most current information.
Tool Calls:
  dashscope_search (call_76c9b6fb13124056bedb0c2d)
 Call ID: call_76c9b6fb13124056bedb0c2d
  Args:
    query: newly appointed President of Bolivia 2024
================================= Tool Message =================================
Name: dashscope_search

As of my last update in October 2023, there is no newly appointed President of Bolivia for the year 2024. The current president of Bolivia is Luis Arce, who took office on November 8, 2020. 

For the most up-to-date information, please check the latest news sources or official government announcements. If there have been any recent changes or elections, th